# MD Simulations

Objectives
- Run a short (100 ns production) MD simulation on each mutant output from PyRosetta.

## Fix PDBs for simulation

In [ ]:
from pathlib import Path
from pdbfixer import PDBFixer
from openmm.app import PDBFile

In [14]:
def fix_pdb(input_path: str | Path, output_path: str | Path | None=None):
    """
    Fix a PDB file using PDBFixer by adding missing residues, atoms, and hydrogens.

    Args:
        input_path (str or Path): Path to the input PDB file.
        output_path (str or Path, optional): Path to save the fixed PDB file. 
            If None, saves as '<input_stem>_fixed.pdb' in the same directory as input.

    Returns:
        None. Writes the fixed PDB file to the specified output path.
    """
    input_path = Path(input_path)
    output_path = Path(output_path) if output_path != None else input_path.parent / (input_path.stem + '_fixed.pdb')
    
    fixer = PDBFixer(filename=str(input_path))
    fixer.findMissingResidues()
    fixer.findMissingAtoms()
    fixer.addMissingAtoms()
    fixer.addMissingHydrogens(pH=7.0)

    with open(output_path, 'w') as f:
        PDBFile.writeFile(fixer.topology, fixer.positions, f)

In [15]:
def fix_all_pdb_in_directory(input_dir: str | Path, output_dir: str | Path):
    """
    Fix all PDB files in a directory using PDBFixer and save them to an output directory.

    Args:
        input_dir (str or Path): Directory containing input PDB files.
        output_dir (str or Path): Directory to save the fixed PDB files. Created if it does not exist.

    Returns:
        None. Writes fixed PDB files with '_fixed.pdb' appended to the filename in the output directory.
    """
    input_dir = Path(input_dir)
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)

    pdb_files = list(input_dir.glob('*.pdb'))

    for pdb in pdb_files:
        output_path = output_dir / (pdb.stem + '_fixed.pdb')
        fix_pdb(pdb, output_path)

In [42]:
fix_all_pdb_in_directory('mutant_library', 'md_sim/inputs')

## Simulation

In [1]:
from pathlib import Path
from openmm.app import (
    PDBFile,
    Modeller,
    ForceField,
    Simulation,
    DCDReporter,
    StateDataReporter,
    CheckpointReporter,
    PME,
    HBonds,
    AllBonds,
    )
from openmm import (
    unit, 
    MonteCarloBarostat, 
    LangevinMiddleIntegrator, 
    Platform
    )
from tqdm import trange
from timeit import default_timer


In [ ]:
# MD Parameters without hbond partitioning
STEP_SIZE      = 4 * unit.femtoseconds
RECORD_EVERY   = 5000                     # steps  (= 20 ps)
NVT_STEPS      = 10000                    # 40 ps
NPT_STEPS      = 25000                    # 100 ps
PROD_STEPS     = 50000                    # 200 ps
TEMPERATURE    = 300 * unit.kelvin
PRESSURE       = 1 * unit.atmosphere
NONBOND_CUTOFF = 1.0 * unit.nanometer
HMASS_PART     = 1.5 * unit.amu
PADDING        = 1.0 * unit.nanometer
SEED           = 42

# https://github.com/openmm/openmm/issues/3695 suggests I can up the step size to 4 fs with HBonds if I set
# hydrogenMass to 1.5 amu.

In [3]:
def run_simulation(
        pdb_path: str | Path,
        output_dir: str | Path,
        step_size: unit.quantity.Quantity = STEP_SIZE,
        record_every: int = RECORD_EVERY,
        nvt_steps: int = NVT_STEPS,
        npt_steps: int = NPT_STEPS,
        prod_steps: int = PROD_STEPS,
        temperature: unit.quantity.Quantity = TEMPERATURE,
        pressure: unit.quantity.Quantity = PRESSURE,
        nonbond_cutoff: unit.quantity.Quantity = NONBOND_CUTOFF,
        padding: unit.quantity.Quantity = PADDING,
        hydrogen_mass: unit.quantity.Quantity | None = HMASS_PART,
        seed: int = SEED
):
    """
    Run a molecular dynamics simulation for a given PDB structure using OpenMM.

    This function performs energy minimization, NVT and NPT equilibration, and a production run.
    Simulation outputs—including trajectory (DCD), log, checkpoint, and final structure (PDB) files—
    are saved to the specified output directory.

    All simulation parameters (step size, temperature, pressure, etc.) are assumed to be predefined
    as global variables/constants, but can be overridden via function arguments.

    Args:
        pdb_path (str or Path): Path to the input PDB file.
        output_dir (str or Path): Directory to save simulation outputs.
        step_size (unit.Quantity, optional): Integration time step.
        record_every (int, optional): Steps between trajectory/log outputs.
        nvt_steps (int, optional): Number of NVT equilibration steps.
        npt_steps (int, optional): Number of NPT equilibration steps.
        prod_steps (int, optional): Number of production steps.
        temperature (unit.Quantity, optional): Simulation temperature.
        pressure (unit.Quantity, optional): Simulation pressure.
        nonbond_cutoff (unit.Quantity, optional): Nonbonded interaction cutoff.
        padding (unit.Quantity, optional): Padding for solvation box.
        hydrogen_mass (unit.Quantity or None, optional): Hydrogen mass repartitioning value.
        seed (int, optional): Random seed for reproducibility.

    Returns:
        None. Writes simulation outputs (DCD, log, checkpoint, final PDB) to output_dir.
    """
    pdb_path = Path(pdb_path)
    pdb_name = pdb_path.stem
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)

    # Load structure and build system
    print(f'[{pdb_name}] Loading structure and building system...')
    pdb = PDBFile(str(pdb_path))
    forcefield = ForceField('amber14-all.xml', 'amber14/opc3.xml')

    modeller = Modeller(pdb.topology, pdb.positions)
    modeller.deleteWater()
    modeller.addHydrogens(forcefield)
    modeller.addExtraParticles(forcefield)
    modeller.addSolvent(forcefield, padding=padding)

    system = forcefield.createSystem(
        modeller.topology,
        nonbondedMethod=PME,
        nonbondedCutoff=nonbond_cutoff,
        constraints=HBonds,
        hydrogenMass=hydrogen_mass
    )

    integrator = LangevinMiddleIntegrator(temperature, 1 / unit.picosecond, step_size)
    integrator.setRandomNumberSeed(seed)

    ## Choose GPU if available
    avail_platforms = [Platform.getName(Platform.getPlatform(idx)) for idx in range(Platform.getNumPlatforms())]
    platform = Platform.getPlatformByName("CUDA") if "CUDA" in avail_platforms else Platform.getPlatformByName("CPU")
    #platform = Platform.getPlatformByName("CPU")

    simulation = Simulation(modeller.topology, system, integrator, platform)
    print(f'[{pdb_name}] Simulation running on platform: {simulation.context.getPlatform().getName()}')

    # Set initial coordinates and minimize
    print(f'[{pdb_name}] Minimizing...')
    simulation.context.setPositions(modeller.positions)
    simulation.minimizeEnergy(
        tolerance=1.0 * unit.kilojoule_per_mole/unit.nanometer,
        maxIterations=5000)
    simulation.context.setVelocitiesToTemperature(temperature)

    # Link up reporters
    print(f'[{pdb_name}] Linking up reporters...')
    simulation.reporters.extend(
        [
            DCDReporter(str(output_dir / f'{pdb_name}.dcd'), record_every),
            StateDataReporter(
                str(output_dir / f'{pdb_name}.log'),
                record_every,
                step=True,
                time=True,
                potentialEnergy=True,
                kineticEnergy=True,
                totalEnergy=True,
                temperature=True,
                speed=True
            ),
            CheckpointReporter(str(output_dir / f'{pdb_name}.chk'), 10000)
        ]
    )

    # NVT equilibration
    print(f'[{pdb_name}] Equilibrating...')
    simulation.step(nvt_steps)

    # NPT equilibration
    system.addForce(MonteCarloBarostat(pressure, temperature))
    simulation.context.reinitialize(preserveState=True)
    simulation.step(npt_steps)

    # Production
    print(f'[{pdb_name}] Simulate!')
    simulation.step(prod_steps)

    # Save final PDB
    state = simulation.context.getState(getPositions=True, getVelocities=True)
    PDBFile.writeFile(
        simulation.topology, state.getPositions(), open(str(output_dir / f"{pdb_name}_final.pdb"), "w")
    )
    print(f'[{pdb_name}] ...and done.')

In [4]:
def simulate_dir(input_dir, output_root):
    """
    Run molecular dynamics simulations for all PDB files in a directory.

    For each PDB file in the input directory, creates a subdirectory in the output root
    and runs a simulation using predefined global simulation parameters (e.g., step size, 
    temperature, pressure, etc.).

    Args:
        input_dir (str or Path): Directory containing input PDB files.
        output_root (str or Path): Root directory where simulation outputs will be stored.

    Returns:
        None. Writes simulation outputs to subdirectories in output_root.

    Notes:
        - Assumes all simulation parameters (such as step size, temperature, etc.) are predefined 
          as global variables/constants in the script.
        - Each simulation output is stored in a separate subdirectory named after the PDB file stem.
    """
    
    input_dir = Path(input_dir)
    output_root = Path(output_root)
    output_root.mkdir(parents=True, exist_ok=True)

    pdb_files = list(input_dir.glob('*.pdb'))

    sim_times = []

    for pdb in pdb_files:
        output_dir = output_root / (pdb.stem.split("_")[0])
        start_time = default_timer()
        run_simulation(pdb, output_dir)
        sim_time = default_timer()-start_time
        print(f'Time taken: {sim_time}')
        sim_times.append(sim_time)
    
    total_time = sum(sim_times)
    avg_time = total_time / len(pdb_files)
    print(f'Total time: {total_time}')
    print(f'Average time per mutant: {avg_time}')

In [5]:
simulate_dir('md_sim/inputs', 'md_sim/outputs')

[I94P-N87C_fixed] Loading structure and building system...
[I94P-N87C_fixed] Simulation running on platform: CUDA
[I94P-N87C_fixed] Minimizing...
[I94P-N87C_fixed] Linking up reporters...
[I94P-N87C_fixed] Equilibrating...
[I94P-N87C_fixed] Simulate!
[I94P-N87C_fixed] ...and done.
Time taken: 119.43264789300001
[Y116F_fixed] Loading structure and building system...
[Y116F_fixed] Simulation running on platform: CUDA
[Y116F_fixed] Minimizing...
[Y116F_fixed] Linking up reporters...
[Y116F_fixed] Equilibrating...
[Y116F_fixed] Simulate!
[Y116F_fixed] ...and done.
Time taken: 100.88712559299998
[D111A-S118R-S117R_fixed] Loading structure and building system...
[D111A-S118R-S117R_fixed] Simulation running on platform: CUDA
[D111A-S118R-S117R_fixed] Minimizing...
[D111A-S118R-S117R_fixed] Linking up reporters...
[D111A-S118R-S117R_fixed] Equilibrating...
[D111A-S118R-S117R_fixed] Simulate!
[D111A-S118R-S117R_fixed] ...and done.
Time taken: 132.697598509
Total time: 353.01737199499996
Average

In [17]:
# GPU without progress bars, 4 fs time step with hbond partitioning
sec_per_ns_per_mutant_gpu = 81
mb_per_mutant = sum([220, 198, 17.2*1024]) / 1024

print(f'Per mutant:\n- {sec_per_ns_per_mutant_gpu} s/ns\n- {mb_per_mutant:.2f} MB')
print(f'Per 1000 mutants:\n- {sec_per_ns_per_mutant_gpu*1000/60/60:.2f} h\n- {mb_per_mutant*1000/1024:.2f} GB')
print(f'Per 10000 mutants:\n- {sec_per_ns_per_mutant_gpu*10000/60/60:.2f} h\n- {mb_per_mutant*10000/1024:.2f} GB')

Per mutant:
- 81 s/ns
- 17.61 MB
Per 1000 mutants:
- 22.50 h
- 17.20 GB
Per 10000 mutants:
- 225.00 h
- 171.96 GB


18 ns/day with cpu, 650 ns/day with gpu

So it would seem that I utterly lack the computational resources to preform the large-scale MD simulations that I desired. I will next look into normal mode analysis, but then I should look into course-grained simulations as well.